<a href="https://colab.research.google.com/github/nafiul-araf/Lab-work/blob/main/Association_rule_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading The Data

In [121]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori,association_rules

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [122]:
df=pd.read_csv("/content/gdrive/MyDrive/CSE-464(Project)/Copy of Depression and Happiness Factor Analysis.csv")
#df.head()

# Preprocessing

In [123]:
df.shape

(920, 21)

In [124]:
df1=df.drop(['Timestamp','How much have you succeeded to cope up with the environment of your educational institution?','How long did you sleep last night?(in hours)','Age','Unnamed: 20'],axis='columns')
#df1.head()

In [125]:
df1.isnull().sum()

Which year are you in?                                                         0
How are you feeling right now?                                                 0
On a scale of 1-100, how would you express this feeling?                       0
Gender                                                                         0
Your location ?                                                                0
Relationship status                                                            0
Are you happy with your financial state?                                       0
Understanding with your family members?                                        0
Are you feeling pressure in your study or work right now?                      0
Are you satisfied with your academic result?                                   0
Are you happy with your living place?                                          0
Who supports you when you are not succeeding in your academic life?            0
Have you used any social med

In [126]:
df1.columns

Index(['Which year are you in?', 'How are you feeling right now?',
       'On a scale of 1-100, how would you express this feeling?', 'Gender',
       'Your location ?', 'Relationship status',
       'Are you happy with your financial state?',
       'Understanding with your family members?',
       'Are you feeling pressure in your study or work right now?',
       'Are you satisfied with your academic result?',
       'Are you happy with your living place?',
       'Who supports you when you are not succeeding in your academic life?',
       'Have you used any social media within the last 6 hours?',
       'Are you satisfied with your meal today?',
       'Are you feeling sick/health issues today?',
       'Have you done any recreational activity (sports, gaming, hobby etc.) today?'],
      dtype='object')

In [127]:
df1.rename({'Relationship status':'Relationship_status','Are you satisfied with your meal today?':'Satisfaction_with_meal','Which year are you in?':'Year','How are you feeling right now?':'Felling_right_now','Your location ?':'Location','Are you happy with your financial state?':'Happy_with_financial','Understanding with your family members?':'Understanding_with_family','Are you feeling pressure in your study or work right now?':'Study_pressure','Are you satisfied with your academic result?':'Satisfaction_academic_result','Are you happy with your living place?':'Living_place_happiness','Who supports you when you are not succeeding in your academic life?':'Supports_in_academic_life','Have you used any social media within the last 6 hours?':'Using_social_media','Do you have inferiority complex?':'Inferiority_complex','Are you feeling sick/health issues today?':'Sick_or_not_today','Have you done any recreational activity (sports, gaming, hobby etc.) today?':'Recreational_activity','On a scale of 1-100, how would you express this feeling?':'Express_feeling'},axis=1,inplace=True)
#df1.head()

In [128]:
df1.shape

(920, 16)

# Implementation

*In this section, I generated the association rules of all the itemsets.*

In [129]:
data=[]

for i in range(0,920):
  data.append([str(df1.values[i,n]) for n in range(0,16)])

In [130]:
from mlxtend.preprocessing import TransactionEncoder

In [131]:
te=TransactionEncoder()

In [132]:
encode=te.fit_transform(data)

encode_df=pd.DataFrame(encode,columns=te.columns_)

In [133]:
#Finding The Frequent Itemsets
freq_itemset=apriori(encode_df,min_support=0.10,use_colnames=True)
freq_itemset

,support,itemsets
0,0.177174,(1)
1,0.232609,(100)
2,0.250000,(1st year)
3,0.250000,(2nd year)
4,0.180435,(30)
...,...,...
1754,0.204348,"(Very good, Yes, Male, Single, Good, 100, Home..."
1755,0.119565,"(Yes, No, Male, Single, 1st year, Hall-Mess, G..."
1756,0.118478,"(Yes, No, Male, 4th year, Single, Family, Home..."
1757,0.131522,"(Yes, No, Male, 4th year, Friends, Single, Hom..."


In [134]:
#Generating The Association Rules
asso_rules=association_rules(freq_itemset,min_threshold=0.06)
asso_rules=asso_rules.sort_values(['support','confidence'])
asso_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1329,(Good),"(Friends, 75)",1.0,0.1,0.1,0.1,1.0,0.0,1.0
1333,(Male),"(Friends, 75)",1.0,0.1,0.1,0.1,1.0,0.0,1.0
1339,(No),"(Friends, 75)",1.0,0.1,0.1,0.1,1.0,0.0,1.0
1345,(Single),"(Friends, 75)",1.0,0.1,0.1,0.1,1.0,0.0,1.0
1351,(Yes),"(Friends, 75)",1.0,0.1,0.1,0.1,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
21939,(Yes),"(Good, Single, Male, No)",1.0,1.0,1.0,1.0,1.0,0.0,inf
21940,(No),"(Single, Male, Yes, Good)",1.0,1.0,1.0,1.0,1.0,0.0,inf
21941,(Male),"(Good, Single, Yes, No)",1.0,1.0,1.0,1.0,1.0,0.0,inf
21942,(Good),"(Single, Male, Yes, No)",1.0,1.0,1.0,1.0,1.0,0.0,inf


In [135]:
asso_rules.shape

(45810, 9)

# Deep Analysis

*In this section, I analyzed the unique element of each features which have more than one unique value or element and found the association rules with my feelings level (Very Good, Good, Normal, Bad, Very Bad)*

**Satisfaction_with_meal**

In [136]:
df1.Satisfaction_with_meal.value_counts()

Yes        490
Neutral    216
No         214
Name: Satisfaction_with_meal, dtype: int64

In [137]:
df1.Satisfaction_with_meal.unique()

array(['Yes', 'Neutral', 'No'], dtype=object)

In [138]:
basket_Yes = (df1[df1['Satisfaction_with_meal'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_Yes

Felling_right_now,Bad,Good,Normal,Very bad,Very good
Year,,,,,
1st year,90.0,3375.0,750.0,0.0,4500.0
2nd year,180.0,2700.0,1140.0,1.0,4985.0
3rd year,0.0,3000.0,1125.0,0.0,4600.0
4th year,451.0,3850.0,1584.0,5.0,6993.0


In [139]:
basket_Neutral = (df1[df1['Satisfaction_with_meal'] =="Neutral"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_Neutral

Felling_right_now,Bad,Good,Normal,Very bad,Very good
Year,,,,,
1st year,510.0,675.0,1749.0,14.0,300.0
2nd year,540.0,300.0,1155.0,7.0,100.0
3rd year,510.0,525.0,1380.0,6.0,0.0
4th year,480.0,150.0,675.0,13.0,100.0


In [140]:
basket_No = (df1[df1['Satisfaction_with_meal'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_No

Felling_right_now,Bad,Good,Normal,Very bad
Year,,,,
1st year,560.0,0.0,570.0,25.0
2nd year,720.0,150.0,405.0,35.0
3rd year,750.0,75.0,360.0,42.0
4th year,150.0,0.0,0.0,14.0


In [141]:
basket_Yes = (df1[df1['Satisfaction_with_meal'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

basket_Neutral = (df1[df1['Satisfaction_with_meal'] =="Neutral"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

basket_No = (df1[df1['Satisfaction_with_meal'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [142]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 
basket_Yes.head()

Felling_right_now,Bad,Good,Normal,Very bad,Very good
Year,,,,,
1st year,1,1,1,0,1
2nd year,1,1,1,1,1
3rd year,0,1,1,0,1
4th year,1,1,1,1,1


In [143]:
basket_encoded = basket_Neutral.applymap(hot_encode) 
basket_Neutral = basket_encoded 
basket_Neutral.head()

Felling_right_now,Bad,Good,Normal,Very bad,Very good
Year,,,,,
1st year,1,1,1,1,1
2nd year,1,1,1,1,1
3rd year,1,1,1,1,0
4th year,1,1,1,1,1


In [144]:
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 
basket_No.head()

Felling_right_now,Bad,Good,Normal,Very bad
Year,,,,
1st year,1,0,1,1
2nd year,1,1,1,1
3rd year,1,1,1,1
4th year,1,0,0,1


In [145]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(Very bad),(Bad),0.5,0.75,0.5,1.0,1.333333,0.125,inf
28,"(Very bad, Good)",(Bad),0.5,0.75,0.5,1.0,1.333333,0.125,inf
30,(Very bad),"(Bad, Good)",0.5,0.75,0.5,1.0,1.333333,0.125,inf
39,"(Normal, Very bad)",(Bad),0.5,0.75,0.5,1.0,1.333333,0.125,inf
43,(Very bad),"(Normal, Bad)",0.5,0.75,0.5,1.0,1.333333,0.125,inf


In [146]:
rules.shape

(180, 9)

***Making Recommendations***

For the first row

In [147]:
basket_Yes['Very bad'].sum()

2

In [148]:
basket_Yes['Bad'].sum()

3

In [149]:
# Building the model 
frq_items = apriori(basket_Neutral, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [150]:
rules.shape

(180, 9)

For the last row

In [151]:
basket_Neutral['Very good'].sum()

3

In [152]:
basket_Neutral['Bad'].sum()

4

In [153]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,(Good),(Normal),0.5,0.75,0.5,1.0,1.333333,0.125,inf
14,"(Bad, Good)",(Normal),0.5,0.75,0.5,1.0,1.333333,0.125,inf
17,(Good),"(Normal, Bad)",0.5,0.75,0.5,1.0,1.333333,0.125,inf
32,"(Very bad, Good)",(Normal),0.5,0.75,0.5,1.0,1.333333,0.125,inf
35,(Good),"(Normal, Very bad)",0.5,0.75,0.5,1.0,1.333333,0.125,inf


For the last row

In [154]:
basket_No[['Normal','Bad']].sum()

Felling_right_now
Normal    3
Bad       4
dtype: int64

In [155]:
basket_No['Very bad'].sum()

4

In [156]:
#df1.head()

**location**

In [157]:
df1.Location.unique()

array(['Home', 'Hall-Mess', 'Department', 'On way to home', 'Playground'],
      dtype=object)

In [158]:
basket_Home = (df1[df1['Location'] =="Home"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_Department = (df1[df1['Location'] =="Department"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_Playground = (df1[df1['Location'] =="Playground"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [159]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Home.applymap(hot_encode) 
basket_Home = basket_encoded 
  
basket_encoded = basket_Department.applymap(hot_encode) 
basket_Department = basket_encoded 
  
basket_encoded = basket_Playground.applymap(hot_encode) 
basket_Playground = basket_encoded 

In [160]:
# Building the model 
frq_items = apriori(basket_Home, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),0.5,1.0,0.5,1.0,1.0,0.0,inf
3,(Bad),(Normal),0.5,1.0,0.5,1.0,1.0,0.0,inf
7,(Bad),(Very good),0.5,1.0,0.5,1.0,1.0,0.0,inf
8,(Normal),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
9,(Good),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [161]:
# Building the model 
frq_items = apriori(basket_Department, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [162]:
# Building the model 
frq_items = apriori(basket_Playground, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(Normal, Very good)",(Good),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
9,(Good),"(Normal, Very good)",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
1,(Good),(Normal),0.333333,0.666667,0.333333,1.0,1.5,0.111111,inf
3,(Good),(Very good),0.333333,0.666667,0.333333,1.0,1.5,0.111111,inf
5,"(Normal, Good)",(Very good),0.333333,0.666667,0.333333,1.0,1.5,0.111111,inf


In [163]:
#df1.head()

***Are you feeling pressure in your study or work right now?***

In [164]:
df1.Study_pressure.unique()

array(['Not applicable', 'Yes', 'No'], dtype=object)

In [165]:
basket_Yes = (df1[df1['Study_pressure'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_No = (df1[df1['Study_pressure'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [166]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 
  
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 

In [167]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [168]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Bad),(Very bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
5,(Very bad),(Bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
27,"(Bad, Good)",(Very bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
28,"(Very bad, Good)",(Bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
29,(Bad),"(Very bad, Good)",0.75,0.75,0.75,1.0,1.333333,0.1875,inf


In [169]:
#df1.head()

**Who supports you when you are not succeeding in your academic life?**

In [170]:
df1.Supports_in_academic_life.unique()

array(['Family', 'Friends', 'Not applicable'], dtype=object)

In [171]:
basket_Family = (df1[df1['Supports_in_academic_life'] =="Family"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_Friends = (df1[df1['Supports_in_academic_life'] =="Friends"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [172]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Family.applymap(hot_encode) 
basket_Family = basket_encoded 
  
basket_encoded = basket_Friends.applymap(hot_encode) 
basket_Friends = basket_encoded 

In [173]:
# Building the model 
frq_items = apriori(basket_Family, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [174]:
# Building the model 
frq_items = apriori(basket_Friends, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [175]:
#df1.head()

**Have you done any recreational activity (sports, gaming, hobby etc.) today?**

In [176]:
df1.Recreational_activity.unique()

array(['No', 'Yes'], dtype=object)

In [177]:
basket_Yes = (df1[df1['Recreational_activity'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_No = (df1[df1['Recreational_activity'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [178]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 
  
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 

In [179]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [180]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [181]:
#df1.head()

**Are you happy with your financial state?**

In [182]:
df1.Happy_with_financial.unique()

array(['Yes'], dtype=object)

In [183]:
basket_Yes = (df1[df1['Happy_with_financial'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [184]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 

In [185]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [186]:
#df1.head()

**Understanding with your family members?**

In [187]:
df1.Understanding_with_family.unique()

array(['Good'], dtype=object)

In [188]:
basket_Good = (df1[df1['Understanding_with_family'] =="Good"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [189]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Good.applymap(hot_encode) 
basket_Good = basket_encoded 

In [190]:
# Building the model 
frq_items = apriori(basket_Good, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [191]:
#df1.head()

**Are you happy with your living place?**

In [192]:
df1.Living_place_happiness.unique()

array(['Yes', 'No', 'Not applicable'], dtype=object)

In [193]:
basket_Yes = (df1[df1['Living_place_happiness'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_No = (df1[df1['Living_place_happiness'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [194]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 
  
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 

In [195]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),0.75,1.0,0.75,1.0,1.0,0.0,inf
3,(Bad),(Normal),0.75,1.0,0.75,1.0,1.0,0.0,inf
4,(Bad),(Very bad),0.75,1.0,0.75,1.0,1.0,0.0,inf
7,(Bad),(Very good),0.75,1.0,0.75,1.0,1.0,0.0,inf
8,(Normal),(Good),1.00,1.0,1.00,1.0,1.0,0.0,inf


In [196]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Normal),(Bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
3,(Bad),(Normal),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
4,(Bad),(Very bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
5,(Very bad),(Bad),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
6,(Very good),(Bad),0.50,0.75,0.50,1.0,1.333333,0.1250,inf


In [197]:
#df1.head()

**Are you satisfied with your academic result?**

In [198]:
df1.Satisfaction_academic_result.unique()

array(['No', 'Not applicable', 'Yes'], dtype=object)

In [199]:
basket_No = (df1[df1['Satisfaction_academic_result'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [200]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 

In [201]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [202]:
#df1.head()

**Have you used any social media within the last 6 hours?**

In [203]:
df1.Using_social_media.unique()

array(['Yes', 'No'], dtype=object)

In [204]:
basket_Yes = (df1[df1['Using_social_media'] =="Yes"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))
basket_No = (df1[df1['Using_social_media'] =="No"] 
          .groupby(['Year', 'Felling_right_now'])['Express_feeling'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Year'))

In [205]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
# Encoding the datasets 
basket_encoded = basket_Yes.applymap(hot_encode) 
basket_Yes = basket_encoded 
  
basket_encoded = basket_No.applymap(hot_encode) 
basket_No = basket_encoded 

In [206]:
# Building the model 
frq_items = apriori(basket_Yes, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [207]:
# Building the model 
frq_items = apriori(basket_No, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head() 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bad),(Good),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(Good),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(Normal),(Bad),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(Bad),(Normal),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(Bad),(Very bad),1.0,1.0,1.0,1.0,1.0,0.0,inf


***The End***